In [ ]:
# google colab

!pip install folium pandas

# Database Request

This section pulls in the data from the database through a GET request.

In [1]:
import http.client
import pandas as pd
import json
import folium
from folium.plugins import MarkerCluster


def database_respone_to_dataframe():
    host = 'apex.oracle.com'
    endpoint = '/pls/apex/hackathonjune2024/NilePProject/GpsData'  
    
    try:
        conn = http.client.HTTPSConnection(host, timeout=10)
        conn.request("GET", endpoint)
        response = conn.getresponse()
        if response.status == 200:
            data = response.read().decode()
            data = json.loads(data)
            return pd.json_normalize(data['items'])
        
    
        else:
            print(f"Error on GET Request: {response.status}, {response.reason}")
        conn.close()
        
    except (ConnectionError, TimeoutError) as e:
        print(f"Error on establishing connection to database: {e}")


df = database_respone_to_dataframe()

# Map Creation

This section is the actual code for the python program. Stable features are implemented here.

In [2]:
def create_line_color(param: str):
    if param == "boat":
        return "blue"
    if param == "plane":
        return "green"
    if param == "truck":
        return "#ff0000"
    if param == "phone":
        return "yellow"


fg = folium.FeatureGroup("Lines") # use to plot lines from coordinates on the map


def create_geolocation(df):
    coords_all = df[["latitude", "longitude"]]
    coords_start = [df.loc[0, "latitude"], df.loc[0, "longitude"]]
    time_start = df.loc[0, "timestamp"]
    coord_last = [df.loc[(len(df) - 1), "latitude"], df.loc[(len(df) - 1), "longitude"]]
    time_last = df.loc[(len(df) - 1), "timestamp"]
    vehicle_last = df.loc[(len(df) - 1), "targetobj"]

        
    basemap = folium.Map(location = coord_last, zoom_start = 13)
    folium.PolyLine(coords_all).add_to(basemap)
    fg.add_to(basemap)
    folium.LayerControl(position='bottomright').add_to(basemap)

    folium.Marker(coords_start,
                  popup = f'Started at: {time_start}',
                  icon=folium.Icon(color='blue')).add_to(basemap)
    folium.Marker(coord_last,
                  popup = f'Last Seen at: {time_last}\nVehicle: {vehicle_last}',
                  icon=folium.Icon(color='red')).add_to(basemap)

    basemap.save("map.html")


create_geolocation(df)

# Presentation

This section is strictly for the presentation with additional info. It includes the test geolocations made with the phone a the university and an artificial forbidden zone for dementration.

In [2]:
df_real = df[df["gpsorigin"]=="real"].copy()
df_mocked = df[df["gpsorigin"]=="mocked"].copy()

df_mocked_sorted = df_mocked.sort_values(by=["timestamp"], ascending=True).copy()
df_mocked_sorted.reset_index(drop=True, inplace=True)

df_mocked_sorted

,ident,gpsid,longitude,latitude,timestamp,targetobj,gpsorigin
0,241,228,33.044217,-0.563281,2024-06-13T13:58:30Z,boat,mocked
1,242,229,32.801322,0.152246,2024-06-13T13:58:31Z,truck,mocked
2,222,211,32.792878,0.176063,2024-06-13T13:58:32Z,truck,mocked
3,224,213,32.803639,0.214565,2024-06-13T13:58:33Z,truck,mocked
4,223,212,32.795913,0.196553,2024-06-13T13:58:33Z,truck,mocked
5,243,230,32.816163,0.251214,2024-06-13T13:58:34Z,truck,mocked
6,225,214,32.812246,0.267419,2024-06-13T13:58:35Z,truck,mocked
7,226,215,32.784884,0.294232,2024-06-13T13:58:36Z,truck,mocked
8,227,216,32.773089,0.312383,2024-06-13T13:58:36Z,truck,mocked
9,228,217,32.761493,0.333583,2024-06-13T13:58:37Z,truck,mocked


In [3]:
def create_line_color(param: str):
    if param == "boat":
        return "blue"
    if param == "plane":
        # return "#008000"
        return "green"
    if param == "truck":
        return "#ff0000"
    if param == "phone":
        return "yellow"


fg = folium.FeatureGroup("Lines") # use to plot lines from coordinates on the map


def create_geolocation(df):
    coords_all = df[["latitude", "longitude"]]
    coords_start = [df.loc[0, "latitude"], df.loc[0, "longitude"]]
    time_start = df.loc[0, "timestamp"]
    coord_last = [df.loc[(len(df) - 1), "latitude"], df.loc[(len(df) - 1), "longitude"]]
    time_last = df.loc[(len(df) - 1), "timestamp"]
    vehicle_last = df.loc[(len(df) - 1), "targetobj"]

        
    basemap = folium.Map(location = coord_last, zoom_start = 13)
    folium.PolyLine(coords_all).add_to(basemap)
    fg.add_to(basemap)
    folium.LayerControl(position='bottomright').add_to(basemap)

    folium.Marker(coords_start,
                  popup = f'Started at: {time_start}',
                  icon=folium.Icon(color='blue')).add_to(basemap)
    folium.Marker(coord_last,
                  popup = f'Last Seen at: {time_last}\nVehicle: {vehicle_last}',
                  icon=folium.Icon(color='red')).add_to(basemap)

    return basemap


map = create_geolocation(df_mocked_sorted)

In [4]:
forbidden_zone = [
    [-0.06558191524279221, 32.34399796251826],
    [-0.11009641489388394, 32.34202385676707],
    [-0.1522581385539316, 32.406183169590214],
    [-0.1320938448167063, 32.415348785707806],
    [-0.06558191524279221, 32.34399796251826]
    ]


folium.PolyLine(forbidden_zone, color='red').add_to(map)

In [5]:
for index, row in df_real.iterrows():
    folium.Marker([row["latitude"], row["longitude"]],
                  popup = f'Time: {row["timestamp"]}\nType: Phone',
                  icon=folium.Icon(color='orange')).add_to(map)

import webbrowser

map.save("map.html")
webbrowser.open("map.html")

True